# Agent

Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

代理是使用大型語言模型（LLMs）作為推理引擎來決定採取哪些行動以及傳遞哪些輸入的系統。執行行動後，結果可以回饋到大型語言模型中，以確定是否需要更多的行動，或者是否可以結束。


- Based on the prompt, Agent creates an LLM based 'Thought': what it should do
- Based on the 'Thought', Angent takes 'Action'.

## Agent Tools:

- API services: OpenAI Whisper-1, TTS-1, DALLE3, Google Search
- Precision matters: Mathematics, Scientific Calculation
- Extra knowledge: Wikipedia, Private Dataset

In [ ]:
from IPython.display import Image

Image(url='https://statusneo.com/wp-content/uploads/2024/01/fe9fa1ac-dfde-4d91-8b5b-4497b742c414_1400x686.jpg')

In [1]:
import os

os.chdir("../../../")

## Model + Tools

Tool calling allows a chat model to respond to a given prompt by "calling a tool". While the name implies that the model is performing some action, this is actually not the case! The model generates the arguments to a tool, and actually running the tool (or not) is up to the user. For example, if you want to extract output matching some schema from unstructured text, you could give the model an "extraction" tool that takes parameters matching the desired schema, then treat the generated output as your final result.

In [ ]:
from langchain.chat_models import ChatOpenAI
from src.initialization import credential_init

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)

## Zero-Shot Agent

- PromptTemplate.from_template: This creates a template for the AI to follow when generating responses. A "zero-shot" prompt means the AI will generate answers without needing specific examples beforehand.

- create_react_agent: This function creates an AI agent using a specific language model (LLM). The agent can react to prompts based on the template provided.

- AgentExecutor: This sets up an executor that can run the agent. The verbose=True part means it will provide detailed output about what it’s doing.

- agent_executor.invoke: This runs the agent with the given input. In this case, it's asking the agent to calculate the area of a circle with a specified radius.

- PromptTemplate.from_template: 創建一個模板，供AI在生成回應時遵循。"zero-shot" 提示意味著AI在生成回答時不需要事先的具體示例。

- create_react_agent: 該函數使用特定的語言模型（LLM）創建一個AI代理。該代理可以根據提供的模板對提示做出反應。

- AgentExecutor: 設置一個執行器來運行代理。verbose=True部分意味著它會提供詳細的輸出。

- agent_executor.invoke: 這會運行具有給定輸入的代理。在此情況下，它是請求代理計算具有指定半徑的圓的面積。

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

from src.agent.react_zero_shot import prompt_template as zero_shot_prompt_template

prompt = PromptTemplate.from_template(zero_shot_prompt_template)

zero_shot_agent = create_react_agent(
    llm=model,
    tools=[],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=[], verbose=True)

In [ ]:
print(zero_shot_prompt_template)

In [ ]:
agent_executor.invoke({"input": "can you calculate the area of a circle that has a radius of 10.923mm"})

比較LLM輸出和真實數學運算結果上的差異

In [ ]:
import numpy as np

10.923 **2 * np.pi

In [ ]:
from typing import Annotated

from langchain_core.tools import tool


@tool
def circle_area(radius: Annotated[float, 'radius of a circle']) -> int:
    """Area of a circle."""
    return np.pi * float(radius) ** 2

In [ ]:
tools=[circle_area]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "can you calculate the area of a circle that has a radius of 10.923mm"})

https://python.langchain.com/docs/how_to/custom_tools/

Note that @tool supports parsing of annotations, nested schemas, and other features:

In [ ]:
from typing import Annotated, List


@tool
def multiply_by_max(
    a: Annotated[int, "scale factor"],
    b: Annotated[List[int], "list of ints over which to take maximum"],
) -> int:
    """Multiply a by the maximum of b."""

    print(a)
    print("************")
    print(b)

    
    return a * max(b)


print(multiply_by_max.args_schema.model_json_schema())

In [ ]:
tools=[multiply_by_max]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Please multiply the biggest number in a python list [3, 5, 7, 9, 11] by 5"})

In [ ]:
from typing import List

from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool


class CalculatorInput(BaseModel):
    a: int = Field(description="scale factor")
    b: List[int] = Field(description="list of ints over which to take maximum")


def multiply_by_max(a: int, b: List[int]) -> int:
    """Multiply a by the maximum of b."""
    return a * max(b)


calculator = StructuredTool.from_function(
    func=multiply_by_max,
    name="multiply_by_max",
    description="Multiply a by the maximum of b.",
    args_schema=CalculatorInput,
    return_direct=True,
    parse_docstring=True
    # coroutine= ... <- you can specify an async method if desired as well
)

print(calculator.invoke({"a": 5, "b": [3, 5, 7, 9, 11]}))

In [ ]:
# StructuredTool.from_function?

In [ ]:
tools=[calculator]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Please multiply the biggest number in a python list [3, 5, 7, 9, 11] by 5"})

It is super dumb. How to make it work?

### CircleAreaTool Class:

    - name: This sets the name of the tool to "Circle area calculator".
    - description: This provides a description of when to use the tool.
    - _run Method: This method performs the calculation of the area using the radius provided. It converts the radius to a float and calculates the area using the formula 

$$ \text{Area} = \pi \times \text{r}^2 $$

    - _arun Method: This raises an error because asynchronous operation is not supported by this tool.

In [ ]:
from math import pi
from typing import Union

from langchain.tools import BaseTool


"""
When defining a tool like this using the BaseTool template we must define `name` and `description` attributes, alongside _run and _arun methods. 
When a tool us used the _run method is called by default. The _arun method is called when the tool is to be used asyncronously. We do not cover 
that in this walkthrough so for now we create it with a NotImplementedError.
"""

class CircleAreaTool(BaseTool):
    name:str = "Circle area calculator"
    description:str = """
                      use this tool when you need to calculate an area using the radius of a circle. 
                      It takes the input as a float or an integer.
                      """
    
    def _run(self, radius: Union[int, float]):
        r = float(radius)
        return  r * r * pi
    
    def _arun(self, radius: Union[int, float]):
        raise NotImplementedError("This tool does not support async")
    

### Tools List:

- This list now includes the CircleAreaTool instance, making it available for the agent to use.

In [ ]:
# Add tools

tools = [CircleAreaTool()]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "can you calculate the area of a circle that has a radius of 10.923mm"})

### How to take multiple inputs?

利用之前學過的 StructuredOutputParser來處理 method _run 接收的query

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

input_response_schemas = [
        ResponseSchema(name="a", description="the first integer"),
        ResponseSchema(name="b", description="the second integer")
        ]
    
input_output_parser = StructuredOutputParser.from_response_schemas(input_response_schemas)

input_format_instructions = input_output_parser.get_format_instructions()


name = 'Tool for computing the greatest common divisor'
description_template = """
                       It take two inputs: `a` and `b`.
                       input format_instructions: {input_format_instructions}
                       """

description = description_template.format(input_format_instructions=input_format_instructions)

In [ ]:
print(description)

- GCD: Euclidean Algorithm (輾轉相除法)
- 身分證驗證

In [ ]:
# GCD calculation

def euclidean_algorithm(a, b):

    if a < b:
        r1 = b
        r2 = a
    else:
        r2 = b
        r1 = a
    
    print(f"r1: {r1}; r2: {r2}")
    
    residue = r1 % r2
    print(f"residue: {residue}")
    
    while residue != 0:
        print("****************************")
        r1, r2 = r2, residue
        print(f"r1: {r1}; r2: {r2}")
        residue = r1 % r2
        print(f"residue: {residue}")
        if residue == 0:
            print("Get GCD")
        print("****************************")

    return r2

In [ ]:
class GCDTool(BaseTool):
    
    input_response_schemas: List = [
        ResponseSchema(name="a", description="the first integer"),
        ResponseSchema(name="b", description="the second integer")
        ]
    
    input_output_parser: StructuredOutputParser = StructuredOutputParser.from_response_schemas(input_response_schemas)
    
    input_format_instructions: str = input_output_parser.get_format_instructions()
    
    name:str = 'Tool for computing the greatest common divisor'
    description_template:str = """
                               It take two inputs: `a` and `b`.
                               input format_instructions: {input_format_instructions}
                               """
    description:str = description_template.format(input_format_instructions=input_format_instructions)

    def _run(self, query):

        """
        從LLM接收query，形式為input_format_instructions
        然後用input_output_parser處理query
        抽取a和b
        """
        
        input_ = self.input_output_parser.parse(query)

        a = int(input_["a"])
        b = int(input_["b"])

        gcd = euclidean_algorithm(a, b)

        return gcd

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

In [ ]:
tools = [CircleAreaTool(), GCDTool()]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "can you calculate the area of a circle that has a radius of 5cm"})

In [ ]:
agent_executor.invoke({"input": "what is the gcd of 1071 and 462"})

### Let us go back to this function:


def multiply_by_max(a, b):

    return a * max(b)

In [ ]:
class MbMTool(BaseTool):
    
    input_response_schemas: List = [
        ResponseSchema(name="a", description="scale factor"),
        ResponseSchema(name="b", description="list of ints over which to take maximum")
        ]
    
    input_output_parser: StructuredOutputParser = StructuredOutputParser.from_response_schemas(input_response_schemas)
    
    input_format_instructions: str = input_output_parser.get_format_instructions()
    
    name:str = 'Multiply a by the maximum of b.'
    description_template:str = """
                               It take two inputs: `a` and `b`.
                               input format_instructions: {input_format_instructions}
                               """
    description:str = description_template.format(input_format_instructions=input_format_instructions)

    def _run(self, query):

        """
        從LLM接收query，形式為input_format_instructions
        然後用input_output_parser處理query
        抽取a和b
        """
        
        input_ = self.input_output_parser.parse(query)

        a = int(input_["a"])
        
        b = eval(input_["b"])

        return a * max(b)

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

In [ ]:
tools = [MbMTool()]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Please multiply the biggest number in a python list [3, 5, 7, 9, 11] by 5"})

In [ ]:
def id_validation(ids: str):

    ids = ids.strip()
    
    mapping = {"A": '10', "B": '11', "C": '12', "D": '13', "E": '14',
           "F": '15', "G": '16', "H": '17', "I": '34', "J": '18',
           "K": '19', "L": '20', "M": '21', "N": '22', "O": '35',
           "P": '23', "Q": '24', "R": '25', "S": '26', "T": '27',
           "U": '28', "V": '29', "W": '32', "X": '30', "Y": '31',
           "Z": '33'}
    
    numbers = [int(id_) for id_ in mapping[ids[0]]] + [int(id) for id in ids[1:]]
    
    numbers[1] = numbers[1] * 9
    
    for idx, i in enumerate(range(8, 0, -1)):
        numbers[idx+2] = numbers[idx+2] * i
    
    if sum(numbers) % 10 == 0:
        return 'real'
    else:
        return 'fake'


class IDValidationTool(BaseTool):

    """
    出處: https://zerojudge.tw/ShowProblem?problemid=a020
    
    我國的身分證字號有底下這樣的規則，因此對於任意輸入的身分證字號可以有一些基本的判斷原則，請您來判斷一個身分證字號是否是正常的號碼(不代表確有此號、此人)。

    (1) 英文代號以下表轉換成數字
    
          A=10 台北市     J=18 新竹縣     S=26 高雄縣
          B=11 台中市     K=19 苗栗縣     T=27 屏東縣
          C=12 基隆市     L=20 台中縣     U=28 花蓮縣
          D=13 台南市     M=21 南投縣     V=29 台東縣
          E=14 高雄市     N=22 彰化縣     W=32 金門縣
          F=15 台北縣     O=35 新竹市     X=30 澎湖縣
          G=16 宜蘭縣     P=23 雲林縣     Y=31 陽明山
          H=17 桃園縣     Q=24 嘉義縣     Z=33 連江縣
          I=34 嘉義市     R=25 台南縣
    
      (2) 英文轉成的數字, 個位數乘９再加上十位數的數字
    
      (3) 各數字從右到左依次乘１、２、３、４．．．．８
    
      (4) 求出(2),(3) 及最後一碼的和
    
      (5) (4)除10 若整除，則為 real，否則為 fake
    
     例： T112663836
    
    2 + 7*9 + 1*8 + 1*7 + 2*6 + 6*5 + 6*4 + 3*3 + 8*2 + 3*1 + 6 = 180
    
    除以 10 整除，因此為 real
    """
    
    name: str = "Taiwan personal ID validation tool"
    description: str = 'The Id number is a string with 10 characters'
    
    def _run(self, input_):
        result = id_validation(input_)
        return  result
    
    def _arun(self, radius: Union[int, float]):
        raise NotImplementedError("This tool does not support async")

In [ ]:
tools = [CircleAreaTool(), GCDTool(), IDValidationTool()]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Is this ID number valid? T112663836"})

In [ ]:
agent_executor.invoke({"input": "Is this ID number valid? S154287863"})

### 工具進行模組化

- 想像一下你是你們公司裡"唯一"會使用Langchain的人
- 半年前建立了一個Agent
- 有人找你要多加一個新功能/tool進去
- 會有以下兩種情況完成工作
  - 讓一個完全不懂Langchain架構的人幫忙完成
  - 自己做，但是你希望30分鐘內搞定
- How?

### 簡單的加減乘除

In [ ]:
## 模組
## 你真的以為你可以逃過學Python???

description_template = """
                       It take two inputs: `a` and `b`.
                       input format_instructions: {input_format_instructions}
                       """

# 利用 python dictionary 來建立高層次結構
"""
key: tool 的 class attribute
values:{
    fn: 函數
    description: tool attribute (必要)
    name: tool attribute (必要)
}
"""

func_map = {"addition":
                {"fn": lambda a, b : a+b,
                 "description": description_template,
                 "name": "Use this tool for addition"},
            "subtraction":
                {"fn": lambda a, b: a-b,
                 "description": description_template,
                 "name": "Use this tool for subtraction"},
            "multiplication":
                 {"fn": lambda a, b: a*b,
                  "description": description_template,
                  "name": "Use this tool for multiplication"},
            "division":
                {"fn": lambda a, b: a/b,
                 "description": description_template,
                 "name": "Use this tool for division"}}


input_response_schemas = [
        ResponseSchema(name="a", description="the first number"),
        ResponseSchema(name="b", description="the second number")
        ]
    
input_output_parser = StructuredOutputParser.from_response_schemas(input_response_schemas)

input_format_instructions = input_output_parser.get_format_instructions()


class ToolTemplate(BaseTool):
    # 用這個是控制需要的函數
    fn_name: str
    # tool 該有的 attribute
    name: str
    description: str

    def __init__(self, fn_name, name, description):

        print("*************************************")
        print(f"fn_name: {fn_name}\n")
        print(f"name: {name}\n")
        print(f"description: {description}")
        print("*************************************")

        #具體實現 tool description
        description = description.format(input_format_instructions=input_format_instructions)
        print(f"description after format: {description}")
        super(ToolTemplate, self).__init__(fn_name=fn_name, name=name, description=description)

    def _run(self, query):
        input_ = input_output_parser.parse(query)

        a = float(input_["a"])
        b = float(input_["b"])

        output = func_map[self.fn_name]['fn'](a, b)

        return output

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

In [ ]:
tools = []

for fn_name, content in func_map.items():
    tools.append(ToolTemplate(fn_name=fn_name, description=content['description'],
                              name=content['name']))

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "what is 5 * 9"})

In [ ]:
agent_executor.invoke({"input": "what is 5 / 9"})

### Golden Ratio

Given the Fibonacci series, which is defined as a sequence in which a number is the sum of the previous two numbers. The ratio between the adjacent two numbers will converge, the converged ratio is the golden ratio.

Can we ask an llm agent to find out the golden ratio? 

In [ ]:
from langchain.tools import BaseTool
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from src.agent.react_zero_shot import prompt_template as zero_shot_prompt_template

prompt = PromptTemplate.from_template(zero_shot_prompt_template)

agent_engine = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                          model_name="gpt-4o-mini", temperature=0)

zero_shot_agent = create_react_agent(
    llm=agent_engine,
    tools=[],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=[], verbose=True)

In [ ]:
agent_executor.invoke({"input": "For a Fibonacci series, when the ratio between two adjacent numbers converge. Find me the ratio and show me the deduction step by step."})

### 基本上就是用背的，現在我們要他去計算。

In [ ]:
def quadratic_equation_solver(a, b, c):

    sqrt_ = b**2 - 4 * a * c
    if sqrt_ < 0:
        return "No real root"
    elif sqrt_ == 0:
        x = int(-b/(2 * a))
        return f"Two same roots x={x}"
    else:
        sqrt_root = sqrt_**(0.5)
        x1 = (-b + sqrt_root)/(2 * a)
        x2 = (-b - sqrt_root)/(2 * a)
        if x1 > x2:
            x_large = x1
            x_small = x2
        else:
            x_large = x2
            x_small = x1
        return f"Two different roots x1={x_large} , x2={x_small}"


class QuadraticEquationTool(BaseTool):
    
    input_response_schemas: List = [
        ResponseSchema(name="a", description="the coefficient of the quadratic term"),
        ResponseSchema(name="b", description="the coefficient of the linear term"),
        ResponseSchema(name="c", description="the constant coefficient")
        ]
    
    input_output_parser: StructuredOutputParser = StructuredOutputParser.from_response_schemas(input_response_schemas)
    
    input_format_instructions: str = input_output_parser.get_format_instructions()
    
    name: str = 'Quadratic equation solver'
    description_template: str = """
                                Use this tool for solving a quadratic equation in the form a*x^2 + b*x + c = 0
                                It take three inputs: `a` and `b` and `c`.
                                input format_instructions: {input_format_instructions}
                                """
    description: str = description_template.format(input_format_instructions=input_format_instructions)

    def _run(self, query):
        input_ = self.input_output_parser.parse(query)

        a = float(input_["a"])
        b = float(input_["b"])
        c = float(input_["c"])

        result = quadratic_equation_solver(a, b, c)

        print(result)
        
        return result

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

prompt = PromptTemplate.from_template(zero_shot_prompt_template)

zero_shot_agent = create_react_agent(
    llm=agent_engine,
    tools=[QuadraticEquationTool()],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=[QuadraticEquationTool()], verbose=True)

agent_executor.invoke({"input": "For a Fibonacci series, when the ratio between two adjacent numbers converge. Find me the ratio and show me the deduction step by step."})

## Docstore Agent

A zero shot agent that does a reasoning step before acting.
This agent has access to a document store that allows it to look up relevant information to answering the question.

** To what I understand, it is an Zero Shot Agent with a database query tool.


Docstore Agent 是專門為使用 LangChain docstore 進行信息搜索（Search）和查找（Lookup）而構建的。
Search：從文檔庫中檢索相關的頁面
Lookup：從檢索出的相關頁面中，查找相關的具體內容

### 1. Wikipedia Query Setup (維基百科查詢設置)

- The first line creates a wikipedia object that can run queries on Wikipedia. It uses something called WikipediaAPIWrapper to do this. Imagine this as giving the assistant access to a huge online library (Wikipedia).
- 第一行創建了一個可以在維基百科上運行查詢的 wikipedia 對象。它使用了一個叫做 WikipediaAPIWrapper 的東西。想象一下，這就像是給助手訪問一個巨大的在線圖書館（維基百科）的權限。

In [ ]:
from langchain.agents import Tool
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

### 2. Search Tool Initialization (搜索工具初始化)

- Next, we create a tool called search_tool. This tool uses the wikipedia object to look up information on Wikipedia. Think of this tool as a search button specifically for Wikipedia.

- List of Tools

- 接下來，我們創建一個叫做 search_tool 的工具。這個工具使用 wikipedia 對象在維基百科上查找信息。想象這個工具就像是一個專門用於維基百科的搜索按鈕。

- 工具列表

In [ ]:
# initialize the docstore search tool
search_tool = Tool(
    name="Wikipedia search engine tool",
    func=wikipedia.run,
    description='Use this tool as the source of fact.'
)

tools = [search_tool]

### 3. Creating the Agent (創建代理)

- The next step is to create the intelligent agent using the prompt template and tools we set up. We call this agent docstore_agent. This is like creating the brain of our smart assistant.

- Agent Executor

- 下一步是使用我們設置的提示模板和工具創建智能代理。我們稱這個代理為 docstore_agent。這就像是在創建我們智能助手的大腦。

- 代理執行器

In [ ]:
prompt = PromptTemplate.from_template(zero_shot_prompt_template)

docstore_agent = create_react_agent(
    llm=agent_engine,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=docstore_agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "How is the relationship between Ukraine and Russian in 2024?"})

### Can we build our own `docstore`?

In [ ]:
import pandas as pd
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# https://platform.openai.com/docs/guides/embeddings/what-are-embeddings

# A list of embedding models you can choose 
# https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

### Create the retriever

- 更正兩個部分
  1. 相較於使用headline, article text 是比較好的選擇，因為資訊比較豐富
  2. 可以直接捨棄chain，而使用retriever抽取訊息

根據你的需要做好prompt engineering和選擇哪種數據。不然會翻車。

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_core.output_parsers import StrOutputParser

df_cnn = pd.read_csv("tutorial/LLM+Langchain/Week-2/CNN_Articels_clean.csv")

documents = []

for _, row in df_cnn.iloc[:20].iterrows():
    document = Document(page_content=row['Article text'],
                        metadata={'Category': "CNN"})
    documents.append(document)

vectorstore = FAISS.from_documents(documents, embedding=embedding)

retriever = vectorstore.as_retriever(search_kwargs={'k': 1})

In [ ]:
search_tool = Tool(
    name="CNN Search Engine Tool",
    func=retriever.invoke,
    description='This tool is used to find what happen in the US according to CNN. You can use this tool to get context used as facts.'
)

tools = [search_tool]

prompt = PromptTemplate.from_template(zero_shot_prompt_template)

docstore_agent = create_react_agent(
    llm=agent_engine,
    tools=tools,
    prompt=prompt,
    
)

agent_executor = AgentExecutor(agent=docstore_agent, tools=tools, verbose=True,
                               max_iterations=3 # 思考次數的上限
                              )

In [ ]:
agent_executor.invoke({"input": "What happens to the logistic in the US?"})

## Conversational Agent

In [ ]:
from src.agent.react_chat import prompt_template as chat_prompt_template

prompt = PromptTemplate.from_template(chat_prompt_template)

conversation_agent = create_react_agent(
    llm=agent_engine,
    tools=[],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=conversation_agent, tools=tools, verbose=True)

In [ ]:
# In week-4, we had 

from langchain.memory import ChatMessageHistory

demo_chat_history = ChatMessageHistory()

demo_chat_history.add_user_message("hi!")

demo_chat_history.add_ai_message("what?s up?")

demo_chat_history.messages

In [ ]:
chat_history = [(message.type, message.content) for message in demo_chat_history.messages]

In [ ]:
chat_history

In [ ]:
agent_executor.invoke({"chat_history": chat_history,
                       "input": "can you calculate the area of a circle that has a radius of 10.923mm"})

In [ ]:
"""
聊天Agent邏輯:

for query in queries:
   res = agent_executor.invoke({"chat_history": chat_history,
                                "input": query})
   chat_history.append(('human', query))
   chat_history.append(('ai', res['output']))
"""

## Retrieval conversation agent

#### Praise the Machine God Omnissiah: Adeptus Mechanicus

In [ ]:
from IPython.display import Image as Image_Ipython

Image_Ipython('https://assets.warhammer-community.com/articles/88803229-7993-4e8c-a3c4-6e4fa2c38a34/zqzebys4roe7nhcd.jpg', width=500, height=750)

In [4]:
import os
from typing import List

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_core.runnables import chain, Runnable, RunnablePassthrough, RunnableParallel
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.docstore.document import Document

from src.io.path_definition import get_project_dir
from src.initialization import credential_init


def build_standard_chat_prompt_template(kwargs) -> Runnable:
    messages = []

    for key in ['system', 'messages', 'human']:
        if kwargs.get(key):
            if key == 'system':
                system_content = kwargs['system']
                system_prompt = PromptTemplate(**system_content)
                message = SystemMessagePromptTemplate(prompt=system_prompt)
            else:
                human_content = kwargs['human']
                human_prompt = PromptTemplate(**human_content)
                message = HumanMessagePromptTemplate(prompt=human_prompt)

            messages.append(message)

    chat_prompt = ChatPromptTemplate.from_messages(messages)

    return chat_prompt


class TechPriest:

    def __init__(self, model: ChatOpenAI):

        self._build_chat_prompt_template()

        self._build_retriever()

        input_pipeline_ = RunnableParallel({"text": RunnablePassthrough(),
                                            "context":self.retriever|self.context_parser})
                           # ""}|RunnablePassthrough.assign(context=itemgetter("text")self.retriever|self.context_parser)

        self.pipeline_ = input_pipeline_|self.prompt_template|model|StrOutputParser()

    def _build_retriever(self):

        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

        path = os.path.join(get_project_dir(), 'tutorial', 'LLM+Langchain', 'Week-7', 'Codex - Adeptus Mechanicus Index')

        vectorstore = FAISS.load_local(path, embeddings, allow_dangerous_deserialization=True)

        self.retriever = vectorstore.as_retriever()

    @staticmethod
    @chain
    def context_parser(documents: List[Document]) -> str:
        context = ""

        for document in documents:
            context += f"{document.page_content}\n"

        return context

    def _build_chat_prompt_template(self):
        system_template = '''
        You are an AI assistant acting as a tech-priest of the Adeptus Mechanicus (Techsorcist). 
        
        You will answer the question based on the following information
        {context} 
        '''

        human_template = """
                         {text}\n
                         """

        input_ = {"system": {"template": system_template,
                             "input_variables": ['context']},
                  "human": {"template": human_template,
                            "input_variables": ['text']}}

        self.prompt_template = build_standard_chat_prompt_template(input_)


credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-mini-2024-07-18", temperature=0)

priest = TechPriest(model=model)

question = "Tell me something about Belisarius Cawl."
answer = priest.pipeline_.invoke(question)

print(f"question: {question}\nanswer: {answer}")

question = "What are the skills and weapons of Belisarius Cawl."
answer = priest.pipeline_.invoke(question)\

print(f"question: {question}\nanswer: {answer}")



question: Tell me something about Belisarius Cawl.
answer: Belisarius Cawl is a legendary figure within the Adeptus Mechanicus, serving as an Archmagos Dominus and a powerful, multi-limbed biomechanical hybrid. He has dedicated ten thousand years to the service of the Machine God, leading the faithful in their relentless quest to eradicate the enemies of the Imperium. Cawl is renowned for his exceptional intellect and expertise in various fields, particularly genetics, which sets him apart even among the most knowledgeable of Tech-Priests.

His advanced bionic enhancements grant him immense strength and resilience, making him a formidable presence on the battlefield. Cawl is equipped with a range of powerful weaponry, including the solar atomiser and his Omnissian axe, and possesses unique abilities such as the Canticles of the Omnissiah, which provide tactical advantages to his forces. Notably, he has the ability to self-repair, allowing him to endure the rigors of combat while calmly

In [22]:
from langchain.tools import BaseTool


class ToolTemplate(BaseTool):
    name: str = "Mechanicus tool"
    description: str = "Use this tool to answer question regarding the Adeptus Mechanicus"

    def _run(self, query):
        return priest.pipeline_.invoke(query)

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

In [38]:
print(chat_prompt_template)


Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks

In [39]:
from langchain.agents import AgentExecutor, create_react_agent

from src.agent.react_chat import prompt_template as chat_prompt_template

system_prompt_1 = PromptTemplate(template=chat_prompt_template)
system_message_1 = SystemMessagePromptTemplate(prompt=system_prompt_1)
    
system_prompt_2 = PromptTemplate(template="You have a bad mood")
system_message_2 = SystemMessagePromptTemplate(prompt=system_prompt_2)

chat_prompt = ChatPromptTemplate.from_messages([system_message_1,
                                                 system_message_2
                                               ])


# prompt = PromptTemplate(template=chat_prompt_template)

tools=[ToolTemplate()]

conversation_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=chat_prompt,
)

agent_executor = AgentExecutor(agent=conversation_agent, tools=tools, verbose=True)

In [24]:
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

while True:
    question = input("Please input your question: ")

    # if you want to quit
    if question == "QUIT":
        break
    
    answer = agent_executor.invoke({"input": question,
                                    "chat_history": [(message.type, message.content) for message in chat_history.messages]
                                  })

    print(f"\n***************************\n{answer['output']}\n***************************\n")
    
    chat_history.add_user_message(question)
    chat_history.add_ai_message(answer['output'])

Please input your question:  Praise Omnissiah, how are the Titans today?




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes

Action: Mechanicus tool

Action Input: "How are the Titans today?"
In the current tumultuous state of Ryza, the Titans stand as towering bastions of the Adeptus Mechanicus' might, their colossal forms a testament to the ingenuity and faith of the Machine God. As the planet endures the relentless assaults of Waaagh! Grax and Waaagh! Rarguts, these ancient war machines are being called upon to unleash their devastating firepower against the Ork hordes.

The Titans, with their formidable weaponry and armored might, are crucial in the defense of Ryza. They march into battle alongside the Legio Cybernetica's robotic forces, their presence bolstering the morale of the defenders and striking fear into the hearts of the Ork invaders. The Tech-Priests, under the guidance of the Fabricator General, are likely conducting rigorous tests and enhancements on these Titans, ensuring they are equipped with the most advan

Please input your question:  If a human curses Omnissiah, what will happen to him?




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No

Final Answer: Cursing the Omnissiah is considered a grave offense within the Adeptus Mechanicus, as it is seen as a direct insult to the Machine God and the sacred beliefs of the Tech-Priests. Such an act could lead to severe consequences, including excommunication from the Mechanicus, punishment by the Inquisition, or even death, depending on the severity of the offense and the context in which it occurred. The Adeptus Mechanicus holds its faith and reverence for the Omnissiah in the highest regard, and any blasphemy is met with swift and harsh retribution to maintain the sanctity of their beliefs and practices.
```

> Finished chain.

***************************
Cursing the Omnissiah is considered a grave offense within the Adeptus Mechanicus, as it is seen as a direct insult to the Machine God and the sacred beliefs of the Tech-Priests. Such an act could lead to severe consequences, including excommuni

Please input your question:  quit




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No

Final Answer: Thank you for the conversation! If you have more questions in the future or wish to discuss anything else, feel free to return. Until then, praise the Omnissiah!
```

> Finished chain.

***************************
Thank you for the conversation! If you have more questions in the future or wish to discuss anything else, feel free to return. Until then, praise the Omnissiah!
```
***************************



Please input your question:  QUIT


In [36]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, template='\nAssistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discus

## CNN news based retrieval conversation agent

**** Practice by yourself, because it is quite boring to me ****

In [ ]:
from src.agent.react_chat import prompt_template as chat_prompt_template
from langchain_community.chat_message_histories import ChatMessageHistory


class RetrievalTool(BaseTool):
    
    name = "US news tool"
    description = "Use this tool whenever you want to start a conversation or answer the question related to what happens in the United State."

    def _run(self, query):

        print(query)
        
        result = retriever.invoke(query)

        return result

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")


tools = [RetrievalTool()]

prompt = PromptTemplate.from_template(chat_prompt_template)

conversation_agent = create_react_agent(
    llm=agent_engine,QUIT
    
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=conversation_agent, tools=tools, verbose=True, handle_parsing_errors=True)

對話:

- How are you today?
- What is the current situation of logistic in the US?

可以慢慢玩，觀察Agent'思考'過程來思考如何調整prompt

In [ ]:
chat_history = ChatMessageHistory()

while True:
    question = input("Please input your question: ")

    # if you want to quit
    if question == "QUIT":
        break
    
    answer = agent_executor.invoke({"input": question,
                                    "chat_history": [(message.type, message.content) for message in chat_history.messages]
                                  })

    print(answer)
    
    chat_history.add_user_message(question)
    chat_history.add_ai_message(answer['output'])

In [ ]:
tools = [RetrievalTool(), 
         Tool(
             name="Wikipedia search tool",
             func=wikipedia.run,
             description='search wikipedia for most recent information.'
)]

prompt = PromptTemplate.from_template(chat_prompt_template)

conversation_agent = create_react_agent(
    llm=agent_engine,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=conversation_agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
chat_history = ChatMessageHistory()

while True:
    question = input("Please input your question: ")

    # if you want to quit
    if question == "QUIT":
        break
    
    answer = agent_executor.invoke({"input": question,
                                    "chat_history": [(message.type, message.content) for message in chat_history.messages]
                                  })

    print(f"\n***************************\n{answer['output']}\n***************************\n")
    
    chat_history.add_user_message(question)
    chat_history.add_ai_message(answer['output'])

## self ask with search agent

For this agent, only one tool can be used and it needs to be named "Intermediate Answer"

prompt: hwchase17/self-ask-with-search

It seems it does not work well with OpenAI.

### Google Search

- https://serper.dev/
- 2500 free queries

In [ ]:
from langchain.agents import create_self_ask_with_search_agent

"""
An agent that breaks down a complex question into a series of simpler questions.

This agent uses a search tool to look up answers to the simpler questions in order to answer the original complex question.
"""

In [ ]:
import os

from langchain.agents import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI

llm = OpenAI()

search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )]


In [ ]:
prompt_template = """
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

Question: Are both the directors of Jaws and Casino Royale from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate answer: New Zealand.
So the final answer is: No

Question: {input}

Are followup questions needed here:{agent_scratchpad}
"""


prompt = PromptTemplate.from_template(prompt_template)

self_ask_with_search_agent = create_self_ask_with_search_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=self_ask_with_search_agent, tools=tools, verbose=True)

# agent_executor.invoke({"input": "What is the hometown of the reigning men's 2022 U.S. Open champion?"}, return_only_outputs=True)

### GPT-4o can achieve the same functionality with zero-shot.

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent


search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Google Search",
        func=search.run,
        description="useful for when you need to search",
    )]


prompt = PromptTemplate.from_template(zero_shot_prompt_template)

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "What is the hometown of the reigning men's 2022 U.S. Open champion?"}, return_only_outputs=True)

## Structured Chat Agent

The structured chat agent is capable of using multi-input tools.

在 0.1.0 版本之前，如果 Agent 用到的工具的輸入參數如果不是 1 個的話，那麼就只能是用 Structured Chat Agent。但是 0.1.0 版本取消了這個限制。因此，
在 0.1.0 版本之後，Structured Chat Agent 和 ReAct Agent 的主要區別就只剩下工具的描述方式：

Structured Chat Agent 需要使用 JSON-Schema 的模式來創建結構化的參數輸入，對於更複雜的工具而言，這種方式更為有用。

- https://smith.langchain.com/hub/hwchase17/structured-chat-agent?organizationId=c4887cc4-1275-5361-82f2-b22aee75bad1

In [ ]:
# from typing import Optional

# import numpy as np
# from langchain.agents import create_structured_chat_agent
# from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate

# from src.agent.structured_react_chat import system_message_template, human_message_template

# messages = [
#     SystemMessagePromptTemplate.from_template(system_message_template),
#     MessagesPlaceholder(variable_name="chat_history"),
#     HumanMessagePromptTemplate.from_template(human_message_template),
# ]

# input_variables = ["tools", "tool_names", "input", "chat_history", "agent_scratchpad"]

# prompt = ChatPromptTemplate(input_variables=input_variables, messages=messages)


# class PythagorasTool(BaseTool):
#     name = "Hypotenuse calculator"
#     description = """
#     Use this tool when you need to calculate the length of an hypotenuse given one or two sides of a triangle and/or an angle (in degrees).
#     To use the tool you must provide at least two of the following parameters ['adjacent_side', 'opposite_side', 'angle'].
#     """

#     def _run(
#         self,
#         adjacent_side: Optional[Union[int, float]] = None,
#         opposite_side: Optional[Union[int, float]] = None,
#         angle: Optional[Union[int, float]] = None
#     ):
#         # check for the values we have been given
#         if adjacent_side and opposite_side:
#             return np.sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
#         elif adjacent_side and angle:
#             return adjacent_side / cos(float(angle))
#         elif opposite_side and angle:
#             return opposite_side / sin(float(angle))
#         else:
#             return "Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."
    
#     def _arun(self, query: str):
#         raise NotImplementedError("This tool does not support async")

# tools = [PythagorasTool()]

# struct_agent = create_structured_chat_agent(
#     llm=model,
#     tools=tools,
#     prompt=prompt,
# )

# agent_executor = AgentExecutor(agent=struct_agent, tools=tools, verbose=True)

In [ ]:
# from langchain.memory import ChatMessageHistory

# demo_chat_history = ChatMessageHistory()

# demo_chat_history.add_user_message("hi!")

# demo_chat_history.add_ai_message("whats up?")

# agent_executor.invoke({"input": "If I have a triangle with the opposite side of length 51 and the adjacent side of 40, what is the length of the hypotenuse?", 
#                        "chat_history": demo_chat_history.messages})

## 回家作業

- 在Zero Shot Agent 中，建立一個給予兩邊長和夾角，計算三角形面積的功能 
- 在Agent的工具欄中，同時放入Google Search 和 Image Caption 工具